In [1]:
import sklearn
import numpy as np
import pandas

In [33]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation, Dropout

In [4]:
from keras.preprocessing.image import ImageDataGenerator

### Try Model from scratch

In [22]:
IMAGE_SIZE = 64
IMAGE_SIZE_TUPLE = (IMAGE_SIZE, IMAGE_SIZE)
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [28]:
batch_size = 32

In [34]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = IMAGE_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
activation_41 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
activation_42 (Activation)   (None, 29, 29, 32)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 12, 12, 64)        18496     
__________

In [35]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['categorical_accuracy'])

In [30]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data/train',
                                                 target_size = IMAGE_SIZE_TUPLE,
                                                 batch_size = batch_size)
val_set = test_datagen.flow_from_directory('data/valid',
                                                 target_size = IMAGE_SIZE_TUPLE,
                                                 batch_size = batch_size)

test_set = test_datagen.flow_from_directory('data/test',
                                            target_size = IMAGE_SIZE_TUPLE,
                                            batch_size = batch_size)

Found 2000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [37]:
history = model.fit_generator(training_set, 
                              steps_per_epoch=2000 // batch_size,
                              epochs = 1,
                              validation_data = val_set,
                              validation_steps=800 // batch_size)


Epoch 1/1
62/62 [==============================] - 376s 6s/step - loss: 0.8114 - categorical_accuracy: 0.6830 - val_loss: 1.2368 - val_categorical_accuracy: 0.5200


### Try Model from pre-trained network on ImageNet

In [38]:
model.evaluate_generator(test_set,steps=100)

KeyboardInterrupt: 